# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [112]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
❌ Error al extraer los datos: HTTPSConnectionPool(host='www.datos.gov.co', port=443): Max retries exceeded with url: /resource/nudc-7mev.json?$limit=50000 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000129411B2450>: Failed to resolve 'www.datos.gov.co' ([Errno 11001] getaddrinfo failed)"))


In [113]:
df_raw

""


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

## Solución 

1. *Limpieza y Preparación*

In [114]:
df_raw.columns = [col.lower() for col in df_raw.columns]

In [115]:
df_raw.columns = df_raw.columns.str.lower()

In [116]:
print(df_raw.columns.tolist())

[]


In [117]:
df_raw.isnull().sum()

Series([], dtype: float64)

In [118]:
print(df_raw.dtypes)

Series([], dtype: object)


In [119]:

columnas_numericas = [
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria',
    'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n',
    'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media',
    'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
    'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria',
    'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media',
    'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
    'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet'
]

# Convertir esas columnas a tipo numérico
for col in columnas_numericas:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')


KeyError: 'poblaci_n_5_16'

In [ ]:

#Se relleno los datos nulos con la media para mantener la estructura de la base de datos 
df_raw[columnas_numericas] = df_raw[columnas_numericas].fillna(df_raw[columnas_numericas].mean())


In [ ]:
print("Años únicos:", df_raw['a_o'].unique())
print("Departamentos únicos:", df_raw['departamento'].nunique())
print("Municipios únicos:", df_raw['municipio'].nunique())

Años únicos: ['2023' '2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015' '2014'
 '2013' '2012' '2011']
Departamentos únicos: 36
Municipios únicos: 1037


In [ ]:

dept_check = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates()
dept_group = dept_check.groupby('c_digo_departamento').agg({'departamento': pd.Series.nunique})
conflictos = dept_group[dept_group['departamento'] > 1]

print(conflictos)

# Error nombre
nombres_conflictivos = df_raw[df_raw['c_digo_departamento'].isin(conflictos.index)][['c_digo_departamento', 'departamento']].drop_duplicates()
print(nombres_conflictivos)

                     departamento
c_digo_departamento              
11                              2
88                              2
     c_digo_departamento                                       departamento
32                    88  Archipiélago de San Andrés, Providencia y Sant...
974                   11                                       Bogotá, D.C.
3389                  88  Archipiélago de San Andrés. Providencia y Sant...
3653                  11                                        Bogotá D.C.


In [ ]:
conflictos_municipio = (
    df_raw.groupby('c_digo_municipio')[['municipio', 'c_digo_departamento']]
    .nunique()
    .query('municipio > 1 or c_digo_departamento > 1')
)

print(conflictos_municipio)

                  municipio  c_digo_departamento
c_digo_municipio                                
11001                     2                    1


In [ ]:
df_raw['departamento'] = df_raw['departamento'].replace({
    'Bogotá D.C.': 'Bogotá, D.C.',
    'Archipiélago de San Andrés. Providencia y Santa Catalina.': 'Archipiélago de San Andrés, Providencia y Santa Catalina'
})

In [ ]:
df_raw.loc[df_raw['c_digo_departamento'] == 88, 'departamento'] = \
    "Archipiélago de San Andrés, Providencia y Santa Catalina"

In [ ]:
df_raw['departamento'] = df_raw['departamento'].str.strip().str.replace(r'[.,;]+', '', regex=True)

In [ ]:
df_raw['municipio'] = df_raw['municipio'].str.strip().str.replace(r'[.,]', '', regex=True)


Cargue Divipola

In [ ]:
import pandas as pd

url = "https://www.datos.gov.co/api/views/gdxc-w37w/rows.csv?accessType=DOWNLOAD"
divipola = pd.read_csv(url)

divipola.head()


,Código Departamento,Nombre Departamento,Código Municipio,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada,longitud,Latitud
0,5,ANTIOQUIA,5001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,5,ANTIOQUIA,5004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"
3,5,ANTIOQUIA,5021,ALEJANDRÍA,Municipio,"-75,141346","6,376061"
4,5,ANTIOQUIA,5030,AMAGÁ,Municipio,"-75,702188","6,038708"


In [ ]:
print(divipola.columns)


Index(['Código Departamento', 'Nombre Departamento', 'Código Municipio',
       'Nombre Municipio', 'Tipo: Municipio / Isla / Área no municipalizada',
       'longitud', 'Latitud'],
      dtype='object')


In [ ]:
divipola = divipola.rename(columns={
    'Código Departamento': 'c_digo_departamento',
    'Nombre Departamento': 'departamento',
    'Código Municipio': 'c_digo_municipio',
    'Nombre Municipio': 'municipio',
    'Tipo: Municipio / Isla / Área no municipalizada': 'tipo_territorial',
    'longitud': 'longitud',
    'Latitud': 'latitud'
})

2. *Creación de Dimensiones* 

In [ ]:
# Convertir a numérico forzando errores como NaN
for col in ['c_digo_departamento', 'c_digo_municipio', 'c_digo_etc']:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

# Eliminar filas con valores faltantes
df_raw = df_raw.dropna(subset=['c_digo_departamento', 'c_digo_municipio', 'c_digo_etc'])

# Convertir a enteros
df_raw['c_digo_departamento'] = df_raw['c_digo_departamento'].astype(int)
df_raw['c_digo_municipio'] = df_raw['c_digo_municipio'].astype(int)
df_raw['c_digo_etc'] = df_raw['c_digo_etc'].astype(int)


In [ ]:
dim_departamento = df_raw[['c_digo_departamento', 'departamento']].drop_duplicates()

dim_departamento = dim_departamento.merge(
    divipola[['c_digo_departamento', 'departamento']].drop_duplicates(),
    on='c_digo_departamento', how='left', suffixes=('', '_divipola')
)

dim_departamento = dim_departamento.reset_index(drop=True)
dim_departamento['id_departamento'] = dim_departamento.index + 1
dim_departamento = dim_departamento[['id_departamento', 'c_digo_departamento', 'departamento']]

# Validación
assert dim_departamento['c_digo_departamento'].is_unique, "Claves duplicadas en dim_departamento"
print("dim_departamento generada correctamente.")



dim_departamento generada correctamente.


In [ ]:

dim_municipio = df_raw[['c_digo_municipio', 'c_digo_departamento']].drop_duplicates()

dim_municipio = dim_municipio.merge(
    divipola[['c_digo_municipio', 'municipio', 'tipo_territorial', 'latitud', 'longitud']],
    on='c_digo_municipio', how='left'
)

dim_municipio = dim_municipio.reset_index(drop=True)
dim_municipio['id_municipio'] = dim_municipio.index + 1

dim_municipio = dim_municipio[['id_municipio', 'c_digo_municipio', 'c_digo_departamento',
                               'municipio', 'tipo_territorial', 'latitud', 'longitud']]

# Validación
assert dim_municipio['c_digo_municipio'].is_unique, "Claves duplicadas en dim_municipio"
print("dim_municipio generada correctamente.")


dim_municipio generada correctamente.


In [ ]:
dim_tiempo = df_raw[['a_o']].drop_duplicates().reset_index(drop=True)
dim_tiempo['id_tiempo'] = dim_tiempo.index + 1
dim_tiempo = dim_tiempo[['id_tiempo', 'a_o']]

# Validación
assert dim_tiempo['a_o'].is_unique, "Años duplicados en dim_tiempo"
print("dim_tiempo generada correctamente.")



dim_tiempo generada correctamente.


3. *Creación de la tabla de Hechos*

In [ ]:
df_hechos = df_raw.copy()

df_hechos = df_hechos.merge(
    dim_departamento[['id_departamento', 'c_digo_departamento']],
    on='c_digo_departamento', how='left'
)

df_hechos = df_hechos.merge(
    dim_municipio[['id_municipio', 'c_digo_municipio']],
    on='c_digo_municipio', how='left'
)

df_hechos = df_hechos.merge(
    dim_tiempo[['id_tiempo', 'a_o']],
    on='a_o', how='left'
)


In [ ]:
columnas_medidas = [
    'tasa_matriculaci_n_5_16', 'cobertura_neta',
    'reprobaci_n_primaria', 'reprobaci_n_secundaria',
    'reprobaci_n_media', 'repitencia'
]

hechos_final = df_hechos[['id_departamento', 'id_municipio', 'id_tiempo'] + columnas_medidas]

print(" Tabla de hechos generada")


 Tabla de hechos generada


In [ ]:
hechos_final.isnull().sum()

id_departamento            0
id_municipio               0
id_tiempo                  0
tasa_matriculaci_n_5_16    0
cobertura_neta             0
reprobaci_n_primaria       0
reprobaci_n_secundaria     0
reprobaci_n_media          0
repitencia                 0
dtype: int64

In [ ]:
hechos_final.describe()


,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
count,14585.000000,14585.000000,14585.000000,14585.000000,14585.000000,14585.00000,14585.000000,14585.000000,14585.000000
mean,18.615632,561.462187,7.000617,84.971929,85.553196,3.88477,6.809006,4.130812,3.299480
std,9.417836,323.883394,3.741392,18.515676,16.867791,3.78979,6.131056,4.190776,3.353208
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,12.000000,281.000000,4.000000,74.940000,76.950000,0.43000,0.760000,0.680000,0.730000
50%,20.000000,561.000000,7.000000,85.200000,86.270000,3.38000,6.210000,3.160000,2.190000
75%,26.000000,842.000000,10.000000,95.300000,94.470000,5.98000,10.790000,6.360000,5.040000
max,34.000000,1123.000000,13.000000,279.030000,264.540000,51.97000,76.970000,67.860000,37.470000


In [ ]:

hechos_final.head()

,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24


In [ ]:
hechos_nombres = hechos_final.copy()

hechos_nombres = hechos_nombres.merge(
    dim_municipio[['id_municipio', 'municipio']],
    on='id_municipio', how='left'
)

hechos_nombres = hechos_nombres.merge(
    dim_departamento[['id_departamento', 'departamento']],
    on='id_departamento', how='left'
)


hechos_nombres = hechos_nombres.merge(
    dim_tiempo[['id_tiempo', 'a_o']],  
    on='id_tiempo', how='left'
)


In [ ]:
hechos_nombres.head()

,id_departamento,id_municipio,id_tiempo,tasa_matriculaci_n_5_16,cobertura_neta,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,municipio,departamento,a_o
0,1,1,1,62.62,62.62,1.96,16.51,2.04,9.52,ABRIAQUÍ,Antioquia,2023
1,2,2,1,53.27,53.27,7.11,9.39,1.75,9.34,EL RETORNO,Guaviare,2023
2,2,3,1,32.52,32.52,6.93,14.13,7.81,8.65,MIRAFLORES,Guaviare,2023
3,3,4,1,59.57,59.57,4.04,8.33,4.60,16.18,MITÚ,Vaupés,2023
4,3,5,1,51.30,51.30,7.32,15.28,7.27,9.24,CARURÚ,Vaupés,2023


## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.

####  Porcentaje de escolaridad por municipio

In [ ]:

porcentaje_escolaridad = hechos_nombres.groupby(['departamento', 'municipio'])['tasa_matriculaci_n_5_16'].mean().reset_index()
porcentaje_escolaridad = porcentaje_escolaridad.rename(columns={'tasa_matriculaci_n_5_16': 'porcentaje_escolaridad'})
porcentaje_escolaridad = porcentaje_escolaridad.sort_values(by='porcentaje_escolaridad', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

porcentaje_escolaridad


,departamento,municipio,porcentaje_escolaridad
517,Cundinamarca,COTA,161.882281
158,Atlántico,PUERTO COLOMBIA,153.048434
545,Cundinamarca,LA CALERA,144.078462
894,Quindio,SALENTO,139.272308
592,Cundinamarca,TENJO,134.685061
528,Cundinamarca,FÚQUENE,128.590769
584,Cundinamarca,SUBACHOQUE,127.242308
970,Santander,PUENTE NACIONAL,125.726923
601,Cundinamarca,VENECIA,124.103077
451,Cesar,EL PASO,122.451538


#### Rendimiento educativo por municipio

In [ ]:
rendimiento = hechos_nombres.groupby(['departamento', 'municipio'])[
    ['repitencia', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media']
].mean().reset_index()


In [ ]:
rendimiento['rendimiento_promedio'] = rendimiento[
    ['repitencia', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media']
].mean(axis=1)


*Departamentos con mejor cobertura*

In [ ]:
mejor_cobertura = hechos_nombres.groupby('departamento')['cobertura_neta'].mean().sort_values(ascending=False)


In [ ]:
!pip install pandasql

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

query = """
SELECT departamento, AVG(cobertura_neta) AS cobertura_promedio
FROM hechos_nombres
GROUP BY departamento
ORDER BY cobertura_promedio DESC
"""

resultado = pysqldf(query)
print(resultado)


                                         departamento  cobertura_promedio
0                                             Quindio           94.579615
1                                               Sucre           93.849172
2                                               Cesar           93.730656
3                                           Magdalena           93.263854
4                                                Meta           90.630439
5                                           Bogotá DC           89.493077
6                                              Tolima           89.239083
7                                        Cundinamarca           88.541257
8                                            Casanare           88.517623
9                                           Antioquia           88.095366
10                                            Córdoba           88.032417
11                                          Atlántico           87.567434
12                                    